# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096814


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:42,  3.64s/it]

Rendering models:  13%|█▎        | 4/30 [00:07<01:07,  2.59s/it]

Rendering models:  17%|█▋        | 5/30 [00:07<00:46,  1.88s/it]

Rendering models:  20%|██        | 6/30 [00:08<00:33,  1.40s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:13<00:00,  4.43it/s]

Rendering models:  93%|█████████▎| 28/30 [00:13<00:00,  4.61it/s]

Rendering models:  97%|█████████▋| 29/30 [00:13<00:00,  4.67it/s]

Rendering models: 100%|██████████| 30/30 [00:14<00:00,  3.39it/s]

memes100                              0.009751
Jnursssmith5263                       0.001721
jnarayanbvg                           0.004626
awright5                              0.001662
jaksonA                               0.491746
ElisabethB                            0.060840
rmz4160                               0.000345
bentleyghioda                         0.000404
not-logged-in-9a45911eb71184fe7c7f    0.000633
AdamMorse                             0.000440
BirdLeaf                              0.001420
cmk24                                 0.000305
not-logged-in-314d48142a3a122ba666    0.008496
not-logged-in-f82a6d15a1dafa9bbfae    0.000294
not-logged-in-4607991ac840eb02289e    0.005086
pangeli5                              0.002398
jzeiszler                             0.000319
AlexYoung35                           0.000497
not-logged-in-d5e7c917502d7f517658    0.230993
41Letters                             0.000346
Planetme                              0.005804
acapirala    

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())